# Term Frequency - Inverse Document Frequency

Notebook Created by [Prashant Brahmbhatt](https://github.com/hashbanger)

________

Valueable references:  

[Christian S. Perone](https://github.com/perone)'s Blog  
[Wikipedia::tf-idf](https://en.wikipedia.org/wiki/Tf%E2%80%93idf)

___

### Understanding Tf-Idf

TF-IDF is used to figure out how important is a word in a document. tf-idf are is a very interesting way to convert the textual representation of information into a **Vector Space Model (VSM)**, or into sparse features.  
VSM is an algebraic model representing textual information as a vector, the components of this vector could represent the importance of a term (tf–idf) or even the absence or presence (Bag of Words) of it in a document.

The first step in modeling the document into a vector space is to create a dictionary of terms present in documents.

We need to select all terms from the document and convert it to a dimension in the vector space, but we would want to remove the **stopwords** first that are present in almost all documents. We want to extract important features from documents, features that could identify them among other similar documents. So using terms like “the, is, at, on”, etc.. is unhelpful.

**Lets us create a toy case for as our data**

In [1]:
train_set = ("The sky is blue.",                          #d1
             "The sun is bright.")                        #d2
test_set = ("The sun in the sky is bright.",              #d3
            "We can see the shining sun, the bright sun.")#d4

We have to create a index vocabulary (dictionary) of the words of the train document set, using the documents **d1** and **d2** from the document set, we’ll have the following index vocabulary denoted as ***E(t)*** where the t is the term:

$$ \mathrm{E}(t) = \begin{cases} 1, & \mbox{if } t\mbox{ is ``blue''} \\ 2, & \mbox{if } t\mbox{ is ``bright''} \\ 3, & \mbox{if } t\mbox{ is ``sky''} \\ 4, & \mbox{if } t\mbox{ is ``sun''} \\ \end{cases}$$

 

We’re going to use the term-frequency to represent each term in our vector space.  
The term-frequency is nothing more than a measure of how many times the terms present in our vocabulary ***E(t)*** are present in the documents **d3** or **d4**, we define the term-frequency as a couting function

$$ \mathrm{tf}(t,d) = \sum\limits_{x\in d} \mathrm{fr}(x, t) $$

where the ***fr(x, t)*** is a simple function defined as:  
$$\mathrm{fr}(x,t) = \begin{cases} 1, & \mbox{if } x = t \\ 0, & \mbox{otherwise} \\ \end{cases} $$

***tf(t, d)*** returns how many times the term **t** is present in the document **d**  
example: ***tf("sun", d4)*** = 2

___

### Creating document vector

Understanding how Tf works, we can move on to the creation of the document vector, which is represented by:  
$$  \displaystyle \vec{v_{d_n}} =(\mathrm{tf}(t_1,d_n), \mathrm{tf}(t_2,d_n), \mathrm{tf}(t_3,d_n), \ldots, \mathrm{tf}(t_n,d_n)) $$

Documents **d3** and **d4** can be represented in vectors as:  
    $$ \vec{v_{d_3}} = (\mathrm{tf}(t_1,d_3), \mathrm{tf}(t_2,d_3), \mathrm{tf}(t_3,d_3), \ldots, \mathrm{tf}(t_n,d_3)) \\ \vec{v_{d_4}} = (\mathrm{tf}(t_1,d_4), \mathrm{tf}(t_2,d_4), \mathrm{tf}(t_3,d_4), \ldots, \mathrm{tf}(t_n,d_4)) $$  
    which evaluates to:  
    $$ \vec{v_{d_3}} = (0, 1, 1, 1) \\ \vec{v_{d_4}} = (0, 1, 0, 2) $$

Here in d4 there is no occurence of the words **blue** and **sky** hence the 0 value.

We have a collection of documents, now represented by vectors, we can represent them as a matrix with **D x F** shape, where **|D|** is the cardinality of the document space, or how many documents we have and the F is the number of features, in our case represented by the vocabulary size.  
$$ M_{|D| \times F} = \begin{bmatrix} 0 & 1 & 1 & 1\\ 0 & 2 & 1 & 0 \end{bmatrix} $$

________

In **sklearn**, what we have presented as the term-frequency, is called **CountVectorizer**

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(stop_words= 'english')

The **CountVectorizer** already uses as default **“analyzer”** called **WordNGramAnalyzer**, which is responsible to convert the *text to lowercase, accents removal, token extraction, filter stop words,* etc

In [3]:
vectorizer.fit_transform(train_set)
print(vectorizer.vocabulary_)

{'sun': 3, 'sky': 2, 'blue': 0, 'bright': 1}


So the vocabulary is same as we supposed in ***E(t)*** except here it begins from 0.

Now using the same vectorizer to create sparse matrix for our **test_set**

In [4]:
test_matrix = vectorizer.transform(test_set)
print(test_matrix)

  (0, 1)	1
  (0, 2)	1
  (0, 3)	1
  (1, 1)	1
  (1, 3)	2


This **test_matrix** is a sparse matrix curretly in **Coordinate Format** but can be coverted into dense format.  
Its dimensions will be as discussed **|D| x F**

In [5]:
test_matrix.todense()

matrix([[0, 1, 1, 1],
        [0, 1, 0, 2]], dtype=int64)

_____

### Inverse Document Frequency

The main problem with the term-frequency approach is that it scales up frequent terms and scales down rare terms which are empirically more informative than the high frequency terms.  
The basic intuition is that a term that occurs frequently in many documents is not a good discriminator.

tf-idf gives is how important is a word to a document in a collection, and that’s why tf-idf incorporates local and global parameters, because it takes in consideration not only the isolated term but also the term within the document collection

tf-idf scales down the frequent terms and scales up the rare occuring words. It does that using a logarithmic scale.    
We can remove stopwords as generally pre defined in the stopwords in library but a better way would be to,  

"convert the entire documents in tf-idf weights and then remove the words with value lower than decided threshold."

Going back to our definition of the **tf(t,d)** which is actually the term count of the term t in the document d.  
The use of this simple term frequency could lead us to problems like **keyword spamming**, which is when we have a repeated term in a document with the purpose of improving its ranking on an IR (Information Retrieval) system or even create a bias towards long documents, making them look more important than they are just because of the high frequency of the term in the document.  

So the term frequency **tf(t,d)** of a document on a vector space is usually also normalized.

_________________________

**----------------------------------------------------------- Optional Maths ----------------------------------------------------------------------------------**

### Vector Normalization

Suppose we want to normalize **d4** - "We can see the shining sun, the bright sun."  
It's vector representation was,  
$$\vec{v_{d_4}} = (0, 1, 0, 2) $$

To normalize the vector, is the same as calculating the *Unit Vector* of the vector, and they are denoted using the **“hat”** notation: **v^ (v hat)**.  
The definition of the unit vector **v^** of a vector **v** is:

$$\displaystyle \hat{v} = \frac{\vec{v}}{\|\vec{v}\|_p} $$

the length of a vector 
$$\vec{u} = (u_1, u_2, u_3, \ldots, u_n)$$
is calculated using the Euclidean norm – a norm is a function that assigns a strictly positive length or size to all vectors in a vector space -, which is defined by:

$$ \|\vec{u}\| = \sqrt{u^2_1 + u^2_2 + u^2_3 + \ldots + u^2_n} $$

We sometimes see that a **p** with vector as with **v** in the unit vector formula. Because it can be generalized as -  
$$ \displaystyle \|\vec{u}\|_p = ( \left|u_1\right|^p + \left|u_2\right|^p + \left|u_3\right|^p + \ldots + \left|u_n\right|^p )^\frac{1}{p} \\  \displaystyle \|\vec{u}\|_p = (\sum\limits_{i=1}^{n}\left|\vec{u}_i\right|^p)^\frac{1}{p} $$

When we read about a **L2-norm**, we’re reading about the Euclidean norm, a norm with **p=2**, the most common norm used to measure the length of a vector, typically called **“magnitude”**; actually, when you have an unqualified length measure (without the p number), you have the L2-norm (Euclidean norm).  

When we read about a **L1-norm**, we’re reading about the norm with **p=1**, defined as:  

$$\displaystyle \|\vec{u}\|_1 = ( \left|u_1\right| + \left|u_2\right| + \left|u_3\right| + \ldots + \left|u_n\right|) $$

Which is nothing more than a simple sum of the components of the vector, also known as Taxicab distance, also called **Manhattan distance.**

$$ \hat{v} = \frac{\vec{v}}{\|\vec{v}\|_p} \\ \\ \hat{v_{d_4}} = \frac{\vec{v_{d_4}}}{||\vec{v_{d_4}}||_2} \\ \\ \\ \hat{v_{d_4}} = \frac{(0,1,0,2)}{\sqrt{0^2 + 2^2 + 1^2 + 0^2}} \\ \\ \hat{v_{d_4}} = \frac{(0,1,0,2)}{\sqrt{5}} \\ \\ \small \hat{v_{d_4}} = (0.0, 0.4472136 , 0.0, 0.89442719) $$

**--------------------------------------------------------------------------------------------------------------------------------------------------------------------**

_________

The cardinality of our document space is defined by $$\left|{D_{train}}\right| = 2 \\ \left|{D_{test}}\right| = 2$$  

since we have only 2 two documents for training and testing, but they obviously don’t need to have the same cardinality.

**Inverse Document frequency (IDF)**is defined as:  
$$ \displaystyle \mathrm{idf}(t) = \log{\frac{\left|D\right|}{1+\left|\{d : t \in d\}\right|}} $$

where $$\left|\{d : t \in d\}\right|$$ is the number of documents where the term t appears, when the term-frequency function satisfies $$ \mathrm{tf}(t,d) \neq 0$$we’re only adding 1 into the formula to avoid zero-division.

So the **Tf-Idf** becomes 
$$\mathrm{tf\mbox{-}idf}(t) = \mathrm{tf}(t, d) \times \mathrm{idf}(t) $$

this formula has an important consequence: a high weight of the tf-idf calculation is reached when you have a high term frequency (tf) in the given document (local parameter) and a low document frequency of the term in the whole collection (global parameter).

Calculating the idf for each feature present in the feature matrix with the term frequency we have calculated in the first tutorial:

M_{train} = $$\begin{bmatrix} 0 & 1 & 1 & 1\\ 0 & 2 & 1 & 0 \end{bmatrix}

Since we have 4 features, we have to calculate 


$$\mathrm{idf}(t_1) = \log{\frac{\left|D\right|}{1+\left|\{d : t_1 \in d\}\right|}} = \log{\frac{2}{1}} = 0.69314718$$

 

$$\mathrm{idf}(t_2) = \log{\frac{\left|D\right|}{1+\left|\{d : t_2 \in d\}\right|}} = \log{\frac{2}{3}} = -0.40546511$$

 

$$\mathrm{idf}(t_3) = \log{\frac{\left|D\right|}{1+\left|\{d : t_3 \in d\}\right|}} = \log{\frac{2}{3}} = -0.40546511$$

 

$$\mathrm{idf}(t_4) = \log{\frac{\left|D\right|}{1+\left|\{d : t_4 \in d\}\right|}} = \log{\frac{2}{2}} = 0.0$$

These idf weights can be represented by a vector as:

$$\vec{idf_{train}} = (0.69314718, -0.40546511, -0.40546511, 0.0) $$